In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mlos_core.optimizers
import ConfigSpace as CS
from ConfigSpace import UniformIntegerHyperparameter
from ConfigSpace import UniformFloatHyperparameter
from ConfigSpace import CategoricalHyperparameter

In [22]:
def cost_model(index):
    performance_data = pd.read_csv('../../build/performance/performance_metrics.csv', header=None, names=["empty_reads", "non_empty_reads", "range_reads", "writes"])
    return performance_data.iloc[index:].mean().sum()

In [38]:
# Start defining a ConfigurationSpace for the Optimizer to search.
input_space = CS.ConfigurationSpace(seed=1234)

max_bytes_for_level_multiplier=CS.Float("max_bytes_for_level_multiplier", (2,20.0), default=10)
max_write_buffer_number=CS.Integer("max_write_buffer_number", (1,20), default=2) #type - integer
avoid_flush_during_shutdown=CS.Float("avoid_flush_during_shutdown", (0,1), default=0) #type - categorical - allowed - true, false

In [39]:
input_space.add_hyperparameters([max_write_buffer_number,max_bytes_for_level_multiplier,avoid_flush_during_shutdown])

[max_write_buffer_number, Type: UniformInteger, Range: [1, 20], Default: 2,
 max_bytes_for_level_multiplier, Type: UniformFloat, Range: [2.0, 20.0], Default: 10.0,
 avoid_flush_during_shutdown, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.0]

In [40]:
#optimizer = mlos_core.optimizers.RandomOptimizer(parameter_space=input_space)

#optimizer = mlos_core.optimizers.FlamlOptimizer(parameter_space=input_space)

optimizer = mlos_core.optimizers.SmacOptimizer(parameter_space=input_space) # , seed=42, n_random_init=20)

In [41]:
optimizer

SmacOptimizer(space_adapter=None)

In [42]:
import os
import time
import sys
def pass_values_to_interface(new_val):
    #print(new_val)
    pipe_path = '../../build/passing_params_pipe'
    if not os.path.exists(pipe_path):
        os.mkfifo(pipe_path)
    pipe_fd = os.open(pipe_path, os.O_RDWR)
    data =''
    try:
        for x in new_val:
            data+='\n' + x + '=' + str(new_val[x].iloc[0])
        data=data.encode('utf-8')
        os.write(pipe_fd, data)
        sys.stdout.flush()
    except KeyboardInterrupt:
        pass
#     finally:
#         os.close(pipe_fd)


In [43]:
def read_epochs_pipe():
    while True:
        epochs_pipe_path = '../../build/passing_epochs'
        if os.path.exists(epochs_pipe_path):
            epochs_pipe_fd = os.open(epochs_pipe_path, os.O_RDONLY)
            try:
                buffer_size = 128
                buffer = os.read(epochs_pipe_fd, buffer_size).decode('utf-8')
                if buffer:
                    reported_epochs = int(buffer)
                    print(f"Reported epochs: {reported_epochs}")
                    return reported_epochs
            except OSError as e:
                if e.errno == os.errno.EAGAIN or e.errno == os.errno.EWOULDBLOCK:
                    pass
                else:
                    raise
            finally:
                os.close(epochs_pipe_fd)
        else:
            pass
    


In [44]:
def run_optimization():
    # get a new config value suggestion to try from the optimizer.
    suggested_value = optimizer.suggest()
    pass_values_to_interface(suggested_value)

    index=read_epochs_pipe()
    time.sleep(5)
    if index==-1:
        target_value=float('inf')
    else:
        target_value = cost_model(index)
    optimizer.register(suggested_value, pd.Series([target_value]))


n_iterations = 3
for i in range(n_iterations):
    run_optimization()

Reported epochs: 7032


IllegalValueError: Value 15.0: (<class 'float'>) is not allowed for hyperparameter max_write_buffer_number, Type: UniformInteger, Range: [1, 20], Default: 2

In [31]:
optimizer.get_observations()

,avoid_flush_during_shutdown,max_bytes_for_level_multiplier,max_write_buffer_number,score
0,0.246147,2.396822,18.446806,236.066667
0,0.312472,17.846580,2.973814,211.800000
0,0.791351,10.323446,11.955038,223.800000


In [32]:
optimizer.get_best_observation()

,avoid_flush_during_shutdown,max_bytes_for_level_multiplier,max_write_buffer_number,score
0,0.312472,17.84658,2.973814,211.8


In [36]:
optimum_value=optimizer.suggest()
pass_values_to_interface(optimum_value)

In [37]:
print(optimum_value)

   avoid_flush_during_shutdown  max_bytes_for_level_multiplier  \
0                     0.104351                       15.493505   

   max_write_buffer_number  
0                13.572928  


In [ ]:
hyper_parameters_integer = {
    "max_open_files":[-1,10000, -1],
    "max_total_wal_size":[0, 100000, 0],
    "delete_obsolete_files_period_micros":[0, 6 * 60 * 60 * 1000000, 6 * 60 * 60 * 1000000],
    "max_background_jobs":[2, 10, 2],
    "max_subcompactions":[1, 100, 1],
    "compaction_readahead_size":[0, 1000000, 0],
    "writable_file_max_buffer_size":[0, 1024 * 1024, 1024 * 1024],
    "delayed_write_rate":[0, 1000000, 0],
    
    "max_write_buffer_number":[1, 10, 2],
    "inplace_update_num_locks":[0, 10000, 0],
    "memtable_huge_page_size":[0, 1000000, 0],
    "arena_block_size":[0, 1000000, 0],
    "level0_slowdown_writes_trigger":[0, 100, 20],
    "level0_stop_writes_trigger":[0, 100, 36],
    "target_file_size_base":[0, 100000000, 64 * 1048576],
    "target_file_size_multiplier":[0, 100, 1],
    "max_compaction_bytes":[0, 1000000000, 64 * 1048576 * 25],
    "soft_pending_compaction_bytes_limit":[0, 1000000000, 64 * 1073741824],
    "hard_pending_compaction_bytes_limit":[0, 1000000000, 256 * 1073741824]                          
}
                                           
hyper_parameters_float ={
    "memtable_prefix_bloom_size_ratio":[0.0, 1.0, 0.0],
    "max_bytes_for_level_multiplier":[0.0, 100.0, 10.0]
}
                                           
hyper_parameters_catagorical ={
    "avoid_flush_during_shutdown":[[True, False],False],  
    "memtable_whole_key_filtering":[[True, False],False],
   "compression_per_level":[["kNoCompression", "kSnappyCompression", "kZlibCompression", "kBZip2Compression", "kLZ4Compression", "kLZ4HCCompression", "kXpressCompression", "kZSTD"], "kNoCompression"]

}